# Entrenamiento de un KNN

En esta hoja de trabajo veremos como elegir los mejores hiperparámetros para entrenar a un KNN, así como entrenar al propio KNN y ver si los hiperparámetros elegidos dan un buen rendimiento.

Empezaremos importando las librerías y funciones que utilizaremos más adelante.

In [1]:
import pandas as pd
import csv
import numpy as np
np.random.seed(357823)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


# Lectura de datos y creación de los atributos y el objetivo

Vamos a empezar por importar el dataset con todos lo atributos que calculamos para la realización de este trabajo. Luego separamos el dataset en atributos por los cuales el KNN se va a entrenar, estos a su vez lo separamos en discretos y continuos para tratarlos mas adelante. Y por último elegiremos el objetivo, en este caso es un objetivo categórico binario.

In [2]:
# Lectura del csv para la tabla con todas las métricas
nodes = pd.read_csv("../tablas/tableWithAllAtributes.csv")
nodes.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [3]:
# Preparación de los datos para la tabla con todos los atributos
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos = nodes.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [4]:
# Elección del objetivo
objetivo = nodes['ml_target']
objetivo.head() 

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

# Tratamiento de los datos

Vamos a tener que codificar la columna name, ya que el modelo KNN no puede trabajar directamente con cadenas de texto. Para ello, utilizaremos la siguiente función para convertir las cadenas en números y luego normalizaremos esos números entre 0 y 1, tomando el máximo y el mínimo para la normalización.

In [5]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)

# Normalizamos el name
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)

# Aplicamos la normalización solo a la columna 'name'
atributos['name'] = normalizador.fit_transform(atributos[['name']])
atributos.head()

///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']


,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,0.061673,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,0.929866,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,0.106687,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,0.191517,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,0.969442,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


# Elección de hiperparámetros

Vamos a elegir los mejores hiperparámetros para entrenar nuestro KNN, para ello vamos a utilizar la búsqueda en rejilla, donde va hacer combinación entre que los veciones sean 1, 3, 5, 7 ó 9 y que la distancia entre ellos se midan o en "manhattan" o "euclidean".

Como verás más adelante nos da que el KNN debería tener un vecino y la distancia debe ser manhattan.

In [6]:
tubería_kNN = Pipeline([
                        ('kNN', KNeighborsClassifier())])
rejilla_de_parámetros = {
    # Número de vecinos impar (tarea de clasificación binaria)
    'kNN__n_neighbors': range(1, 10, 2),
    # Considerar las distancias Manhattan y euclídea
    'kNN__metric': ['manhattan', 'euclidean']
}

In [7]:
búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos, objetivo)

GridSearchCV(cv=10, estimator=Pipeline(steps=[('kNN', KNeighborsClassifier())]),
             param_grid={'kNN__metric': ['manhattan', 'euclidean'],
                         'kNN__n_neighbors': range(1, 10, 2)},
             scoring='recall')

In [8]:
búsqueda_en_rejilla.best_params_

{'kNN__metric': 'manhattan', 'kNN__n_neighbors': 1}

In [9]:
búsqueda_en_rejilla.best_score_

0.5438398357289527

# Validación por retención

Ahora vamos a comprobar si los resultados de la búsqueda en rejilla son acertados, para ello vamos a dividir el dataset en datos de entrenamiento y datos de pruebas, en este caso vamos a dividir en un 80% de los datos para entrenar y un 20% para comprobar cómo se ha entrenado.

Para ello vamos a entrenar varios KNN, cambiando los hiperparámetros y ver cual nos da un mejor rendimiento y luego analizaremos qué significan esos rendimientos.

In [10]:
# Dividimos los conjuntos de prueba y entrenamiento
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (30 % en este caso)
        test_size=.2, # El más óptimo es con el 30% de conjunto de prueba
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo,
        random_state=357823)

In [11]:
# KNN con 1 vecino y distancia manhattan
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 1 ejemplos más cercanos
    n_neighbors=1,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)

KNeighborsClassifier(metric='manhattan', n_neighbors=1)

In [12]:
predicciones = clasificador_kNN.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 1., 0.])

In [13]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Negativo", "Verdadero Positivo"], columns=["Predicho Negativo", "Predicho Positivo"])
print(tabla_cm)

                    Predicho Negativo  Predicho Positivo
Verdadero Negativo               4191               1401
Verdadero Positivo               1421                527


In [14]:
recall_score(objetivo_prueba, predicciones)

0.2705338809034908

In [15]:
# KNN con 3 vecinos y distancia manhattan
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 1 ejemplos más cercanos
    n_neighbors=3,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)

KNeighborsClassifier(metric='manhattan', n_neighbors=3)

In [16]:
predicciones = clasificador_kNN.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 0., 0.])

In [17]:
cm = confusion_matrix(objetivo_prueba,predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Negativo", "Verdadero Positivo"], columns=["Predicho Negativo", "Predicho Positivo"])
print(tabla_cm)

                    Predicho Negativo  Predicho Positivo
Verdadero Negativo               4656                936
Verdadero Positivo               1593                355


In [18]:
recall_score(objetivo_prueba, predicciones)

0.1822381930184805

In [19]:
# KNN con 1 vecino y distancia euclidea
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 1 ejemplos más cercanos
    n_neighbors=1,
    # La cercanía viene determinada por la distancia euclídea
    metric='euclidean'
)
clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)

KNeighborsClassifier(metric='euclidean', n_neighbors=1)

In [20]:
predicciones = clasificador_kNN.predict(atributos_prueba)
predicciones

array([0., 0., 0., ..., 0., 1., 0.])

In [21]:
cm = confusion_matrix(objetivo_prueba, predicciones)
tabla_cm = pd.DataFrame(cm, index=["Verdadero Positivo", "Verdadero Negativo"], columns=["Predicho Positivo", "Predicho Negativo"])
print(tabla_cm)

                    Predicho Positivo  Predicho Negativo
Verdadero Positivo               4207               1385
Verdadero Negativo               1430                518


In [22]:
recall_score(objetivo_prueba, predicciones)

0.26591375770020537

# Análisis del resultado

Al observar nuestros resultados, notamos que el mejor puntaje se obtiene al establecer un vecino y calcular la distancia utilizando el método de Manhattan. Esto nos proporciona una confianza del 26.59%. Dado que en nuestros datos el 75% corresponden a desarrolladores web y el 25% a desarrolladores de IA, al examinar la matriz de confusión, concluimos que el KNN es menos efectivo que un clasificador aleatorio. Esto se debe a que predice demasiados casos como positivos, lo que podría ser resultado de un desequilibrio en la distribución de clases en los datos de entrenamiento. Al haber más casos de desarrolladores web, es probable que el modelo haya aprendido que la mayoría de las instancias son de ese tipo, lo que se refleja en la alta cantidad de positivos predichos y la escasa cantidad de negativos.